In [1]:
from time import strftime

import torch as t
import os
import torchvision as tv
from torch.utils.data import DataLoader
import numpy as np
from torch import nn
from torch.optim import Adam
import tqdm


import models
from attack.PGDAttack import LinfPGDAttack
DOWNLOAD_CIFAR = False #是否下载数据集

In [2]:

def train():
    '''
    训练神经网络
    :return:
    '''
    #1.加载配置
    opt = Config()
    opt._parese()
    global DOWNLOAD_CIFAR

    #1a.加载模型
    model = getattr(models,opt.model)()
    if opt.model_path:
        model.load(opt.model_path)
    model.to(opt.device)

    #2.定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=opt.lr)


    #3.加载数据
    if not (os.path.exists('./data/CIFAR/')) or not os.listdir('./data/CIFAR/'):
        DOWNLOAD_CIFAR =True

    transform = tv.transforms.Compose([
        #要先完成所有操作，然后totensor，normalize
        tv.transforms.ToTensor(),
        tv.transforms.Normalize(mean=opt.MEAN, std=opt.STD)
    ])
    train_data = tv.datasets.CIFAR10(
        root='./data/CIFAR/',
        train=True,
        transform=transform,
        download=DOWNLOAD_CIFAR
    )

    train_loader = DataLoader(
        dataset=train_data,
        batch_size=opt.batch_size,
        shuffle=True,
        num_workers=opt.num_workers
                              )

    test_data = tv.datasets.CIFAR10(
        root='./data/CIFAR/',
        train = False,
        transform=transform,
        download=DOWNLOAD_CIFAR
    )
    # test_data.loader
    test_loader = DataLoader(test_data, batch_size=opt.test_num, shuffle=True, num_workers=opt.num_workers)

    #训练模型
    for epoch in range(opt.train_epoch):
        for ii,(data,label) in tqdm.tqdm(enumerate(train_loader)):
            input = data.to(opt.device)
            target = label.to(opt.device)

            optimizer.zero_grad()
            score = model(input)
            loss = criterion(score,target)
            loss.backward()
            optimizer.step()

            if (ii+1)%opt.print_freq ==0:
                print('loss:%.2f'%loss.cpu().data.numpy())
        if (epoch + 1) % opt.save_every == 0:
            model.save(epoch=epoch+1)

            #测试
            for ii, (test_x, test_y) in enumerate(test_loader):
                test_x = test_x.to(opt.device)
                test_score = model(test_x)
                correct = np.sum((t.argmax(test_score.to('cpu'), 1) == test_y).numpy())
                print('accuracy: {} '.format(round(correct/test_y.size(0),4)))
                string = 'model: {} | data: natural | accuracy: {} | epoch: {} \n'.format(opt.model, round(correct/test_y.size(0),4), epoch+1)
                open(opt.writeFile,'a').write(string)
                break

@t.no_grad()
def test_acc():
    #测试准确率
    # 1. 加载配置
    opt = Config()
    opt._parese()
    global DOWNLOAD_CIFAR
    if not (os.path.exists('./data/CIFAR/')) or not os.listdir('./data/CIFAR/'):
        DOWNLOAD_CIFAR=True

    # 1a.加载模型
    model = getattr(models, opt.model)()
    if opt.model_path:
        model.load(opt.model_path)
    model.to(opt.device)
    model.eval()

    # 2.加载数据
    transform = tv.transforms.Compose([
        tv.transforms.ToTensor(),
        tv.transforms.Normalize(mean=opt.MEAN, std=opt.STD)

    ])

    test_data = tv.datasets.CIFAR10(
        root='./data/CIFAR/',
        train = False,
        transform=transform,
        download=DOWNLOAD_CIFAR
    )
    #test_data.loader
    test_loader = DataLoader(test_data, batch_size=1, shuffle=True, num_workers=opt.num_workers)

    correct_num = 0
    for ii, (test_x, test_y) in enumerate(test_loader):
        if ii >= opt.test_num:
            break
        test_x = test_x.to(opt.device)
        test_score = model(test_x)
        correct = (t.argmax(test_score.to('cpu'), 1) == test_y).numpy()
        correct_num = correct_num + correct[0]

    accuracy = correct_num/opt.test_num
    print('test accuracy:%.2f' % accuracy)
    return accuracy


def train_adv_PGD():
    '''
    使用PGD的生成样本进行训练
    :return:
    '''

    # 1.加载配置
    opt = Config()
    opt._parese()
    global DOWNLOAD_CIFAR
    if not (os.path.exists('./data/CIFAR/')) or not os.listdir('./data/CIFAR/'):
        DOWNLOAD_CIFAR=True

    # 1a.加载模型
    natural_model = getattr(models, opt.model)()
    adv_model = getattr(models, opt.model)()
    if opt.model_path:
        natural_model.load(opt.model_path)
    natural_model.to(opt.device)
    
    if opt.model_path_adv:
        adv_model.load(opt.model_path_adv)
    adv_model.to(opt.device)

    #2.定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(adv_model.parameters(), lr=opt.lr)

    # 3.加载数据
    transform = tv.transforms.Compose([
        tv.transforms.ToTensor(),
        tv.transforms.Normalize(mean=opt.MEAN,std= opt.STD)
    ])

    train_data = tv.datasets.CIFAR10(
        root='./data/CIFAR/',
        train=True,
        transform=transform,
        download=DOWNLOAD_CIFAR
    )

    train_loader = DataLoader(
        dataset=train_data,
        batch_size=opt.batch_size,
        shuffle=True,
        num_workers=opt.num_workers
    )
    test_data = tv.datasets.CIFAR10(
        root='./data/CIFAR/',
        train = False,
        transform=transform,
        download=DOWNLOAD_CIFAR
    )
    # test_data.loader
    test_loader = DataLoader(test_data, batch_size=opt.test_num, shuffle=True, num_workers=opt.num_workers)

    attack = LinfPGDAttack(natural_model,opt.epsilon)
    # 训练模型
    for epoch in range(opt.train_epoch):
        for ii, (data, label) in tqdm.tqdm(enumerate(train_loader)):
            optimizer.zero_grad()

            perturb_x = attack.perturb(data.numpy(), label)

            score = adv_model(t.FloatTensor(perturb_x).to(opt.device))
            loss = criterion(score, label.to(opt.device))
            loss.backward()
            optimizer.step()

            if (ii + 1) % opt.print_freq == 0:
                print('loss:%.2f' % loss.cpu().data.numpy())
        if (epoch + 1) % opt.save_every == 0:
            adv_model.save(epoch=epoch + 1,isAdv=True)

            #测试
            for ii, (test_x, test_y) in enumerate(test_loader):
                test_x = test_x.to(opt.device)
                test_score = adv_model(test_x)
                correct = np.sum((t.argmax(test_score.to('cpu'), 1) == test_y).numpy())
                print('accuracy: {} '.format(round(correct/test_y.size(0),4)))
                string = 'model: {} | data: adv | accuracy: {} | epoch: {} \n'.format(opt.model, round(correct/test_y.size(0),4), epoch+1)
                open(opt.writeFile,'a').write(string)
                break


In [3]:
class Config:
    model_path = None# 预训练模型，None表示重新训练
    model = 'ShuffleNetV2'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train()

Caculate on cuda
user config:
model_path None
model ShuffleNetV2
epsilon 0.3
writeFile log.txt
lr 0.0005
use_gpu True
MEAN (0.4914, 0.4822, 0.4465)
STD (0.2023, 0.1994, 0.201)
train_epoch 200
save_every 20
test_num 16
batch_size 128
print_freq 500
num_workers 8


391it [00:22, 17.64it/s]
391it [00:22, 17.52it/s]
391it [00:21, 18.61it/s]
391it [00:21, 18.14it/s]
391it [00:22, 17.28it/s]
391it [00:22, 17.39it/s]
391it [00:21, 17.85it/s]
391it [00:23, 16.91it/s]
391it [00:21, 17.98it/s]
391it [00:21, 18.17it/s]
391it [00:21, 18.57it/s]
391it [00:21, 17.83it/s]
391it [00:21, 18.48it/s]
391it [00:21, 18.02it/s]
391it [00:23, 16.51it/s]
391it [00:21, 17.90it/s]
391it [00:22, 17.46it/s]
391it [00:20, 18.65it/s]
391it [00:21, 17.90it/s]
391it [00:22, 17.51it/s]


model saved at ./ckps/ShuffleNetV2_ep20.pth
accuracy: 0.5625 


391it [00:22, 17.35it/s]
391it [00:21, 18.06it/s]
391it [00:20, 18.80it/s]
391it [00:19, 19.75it/s]
391it [00:20, 19.35it/s]
391it [00:19, 19.76it/s]
391it [00:20, 19.32it/s]
391it [00:20, 19.04it/s]
391it [00:19, 20.57it/s]
391it [00:19, 19.89it/s]
391it [00:20, 19.07it/s]
391it [00:21, 18.38it/s]
391it [00:20, 19.32it/s]
391it [00:19, 19.66it/s]
391it [00:20, 19.04it/s]
391it [00:20, 19.23it/s]
391it [00:20, 19.55it/s]
391it [00:20, 18.86it/s]
391it [00:20, 19.41it/s]
391it [00:21, 18.39it/s]


model saved at ./ckps/ShuffleNetV2_ep40.pth
accuracy: 0.5 


...

391it [00:23, 16.77it/s]
391it [00:22, 17.77it/s]
391it [00:21, 18.59it/s]
391it [00:21, 18.40it/s]
391it [00:21, 18.12it/s]
391it [00:22, 17.69it/s]
391it [00:22, 17.36it/s]
391it [00:21, 18.06it/s]
391it [00:22, 17.76it/s]
391it [00:22, 17.36it/s]
391it [00:22, 17.45it/s]
391it [00:20, 19.33it/s]
391it [00:21, 18.21it/s]
391it [00:21, 17.83it/s]
391it [00:21, 18.00it/s]
391it [00:21, 17.91it/s]
391it [00:21, 18.11it/s]
391it [00:21, 18.14it/s]
391it [00:21, 18.08it/s]
391it [00:22, 17.68it/s]

model saved at ./ckps/ShuffleNetV2_ep180.pth


accuracy: 0.5625 


391it [00:21, 17.98it/s]
391it [00:22, 17.34it/s]
391it [00:22, 17.49it/s]
391it [00:21, 18.21it/s]
391it [00:22, 17.08it/s]
391it [00:21, 18.10it/s]
391it [00:22, 17.52it/s]
391it [00:21, 18.48it/s]
391it [00:23, 16.66it/s]
391it [00:21, 17.98it/s]
391it [00:23, 16.64it/s]
391it [00:21, 18.24it/s]
391it [00:22, 17.66it/s]
391it [00:21, 17.79it/s]
391it [00:23, 16.66it/s]
391it [00:22, 17.19it/s]
391it [00:21, 18.06it/s]
391it [00:21, 18.24it/s]
391it [00:19, 19.87it/s]
391it [00:19, 20.00it/s]


model saved at ./ckps/ShuffleNetV2_ep200.pth
accuracy: 0.5625 


In [ ]:
class Config:
    model_path = 'ShuffleNetV2_ep200.pth'# 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'ShuffleNetV2'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train_adv_PGD()

Caculate on cuda
user config:
model_path ShuffleNetV2_ep200.pth
model_path_adv None
model ShuffleNetV2
epsilon 0.3
writeFile log.txt
lr 0.0005
use_gpu True
MEAN (0.4914, 0.4822, 0.4465)
STD (0.2023, 0.1994, 0.201)
train_epoch 200
save_every 20
test_num 16
batch_size 128
print_freq 500
num_workers 8


391it [10:05,  1.55s/it]
391it [10:09,  1.56s/it]
391it [10:11,  1.56s/it]
391it [10:09,  1.56s/it]
391it [10:08,  1.56s/it]
391it [10:11,  1.56s/it]
391it [09:58,  1.53s/it]
391it [09:17,  1.43s/it]
391it [09:21,  1.44s/it]
391it [09:27,  1.45s/it]
391it [09:17,  1.42s/it]
185it [04:26,  1.53s/it]

In [ ]:
class Config:
    model_path = None# 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'ShuffleNetV2_x2'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train()

In [ ]:
class Config:
    model_path = 'ShuffleNetV2_x2_ep200.pth'# 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'ShuffleNetV2_x2'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train_adv_PGD()

In [ ]:
class Config:
    model_path = None # 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'ShuffleNetV2_x4'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train()

In [ ]:
class Config:
    model_path = "ShuffleNetV2_x4_ep200.pth" # 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'ShuffleNetV2_x4'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train_adv_PGD()

In [ ]:
class Config:
    model_path = None # 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'MobileNetV2'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train()

In [ ]:
class Config:
    model_path = 'MobileNetV2_ep200.pth' # 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'MobileNetV2'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train_adv_PGD()

In [ ]:
class Config:
    model_path = None # 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'MobileNetV2_x2'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train()

In [ ]:
class Config:
    model_path = 'MobileNetV2_x2_ep200.pth' # 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'MobileNetV2_x2'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train_adv_PGD()

In [ ]:
class Config:
    model_path = None # 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'MobileNetV2_x4'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train()

In [ ]:
class Config:
    model_path = 'MobileNetV2_x4_ep200.pth' # 预训练模型，None表示重新训练
    model_path_adv = None #用生成对抗样本训练的模型，None表示重新训练
    model = 'MobileNetV2_x4'#加载的模型，模型名必须与models/__init__.py中的名字一致
    epsilon = 0.3   #PGD攻击中的干扰参数
    writeFile = 'log.txt'
    '''
    ShuffleNetV2, ShuffleNetV2_x2, 
    ShuffleNetV2_x4, MobileNetV2, 
    MobileNetV2_x2,MobileNetV2_x4,
    '''
    lr = 0.0005 #学习率
    use_gpu = True  #是否使用gpu
    MEAN= (0.4914, 0.4822, 0.4465)
    STD=(0.2023, 0.1994, 0.2010)#均值和方差
    train_epoch = 200  # 将数据集训练多少次
    save_every = 20  # 每训练多少轮保存一次模型
    # imagenet得出的较好的值，具体过程参考
    # https://cloud.tencent.com/developer/ask/153881

    test_num = 16  # 选择攻击和测试的样本数量


    batch_size = 128  # 每次喂入多少数据

    print_freq = 500  # 每训练多少批次就打印一次
    num_workers = 8 #加载数据集的线程数

    def _parese(self):
        self.device = t.device('cuda') if self.use_gpu else t.device('cpu')
        print('Caculate on {}'.format(self.device))
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('_'):
                print(k, getattr(self, k))
train_adv_PGD()